## Step 1: Start State
- **State Number**: 1
- **State Code**: 
- **State Name**: Start State
- **Description**: The process begins here when the machine is triggered. All attributes of the contract and the new subscriber group are gathered to determine the next state.
- **Status**: Pending


## Step 2: K2 DST Subscriber Group Change Initiated - Change Process Control
- **State Number**: 2
- **State Code**: 
- **State Name**: K2 DST Subscriber Group Change Initiated - Change Process Control
- **Description**: This state performs the necessary checks related to the contract and installation. The change process is initiated here.
- **Status**: Pending



#### 2A - Çalışma Öncesi Scripti

#### 2B - İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do business/mapping here
    return {"body": {}, "params": {"assetId":str(request_data["id"])}}


#### 2C - State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    
    attribute_codes = ['ServicePointId', 'ContractAccountId']
    
    for code in attribute_codes:
        for attr in request_data['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # values içindeki value al
                store[code] = value
                break
        else:
            store[code] = None  # Eğer bulunamazsa None döner
    return store

#### 2D - Çalışma Sonrası Scripti

In [1]:
# machine                     : Currently executing machine instance

# state                       : Currently executing state instance

# request_data                : Original trigger data

# context                     : Machine stored context

# state_responses_of_machine  : State responses of Machine <StateCode, {prev, businessLogic, webhook}>

# return

#     {"success": True, "goto": "STATE_CODE"}

#     {"success": False, "error": {"code": "...", "detail": "..."}}

def state_handler(machine, state, request_data, context, state_responses_of_machine):

    contract_status=False

    energy_status=False

    code = state['stateDefinition']['stateCode']

    prev_state_response = state_responses_of_machine[code]['businessLogicResponseOfPreviousState']
 
    for attr in request_data['attributes']:

        if attr['attribute']['attributeCode'] == 'ContractCancellationDate' and attr['value'] == 99991231235959:

            contract_status=True
 
    for attr in prev_state_response['attributes']:

            if attr['attribute']['attributeCode'] == 'EnergyStatus' and attr['value'] == 'OPEN':

                energy_status=True

    if contract_status and energy_status:

        return {"success": True, "goto": "10380003"}

    if contract_status and energy_status == False:

        return {"success": True, "goto": "10380004"}

    # do workflow here

    return {"success": True, "goto": "END"}
 

#### 2E - İş mantığı

In [ ]:
"""md/search"""

## Step 3: Period Invoice Creation
- **State Number**: 3
- **State Code**: 
- **State Name**: Period Invoice Creation
- **Description**: A period invoice is created based on the first period reading. This step awaits a Kafka event (`thor-billing-accrual-created`) for the invoice creation.
- **Status**: Pending


## Step 4: Contract Termination
- **State Number**: 4
- **State Code**: 
- **State Name**: Contract Termination
- **Description**: After the period invoice is created, the contract is terminated. Historical date attributes are set according to the invoice creation date.
- **Status**: Pending


#### 4A - Çalışma Öncesi Scripti

In [ ]:
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    return {}

#### 4B - İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": next((attr['value'] for attr in request_data['attributes'] 
                               if attr['attribute']['attributeCode'] == "ContractNumber"), None)
            }
        ],
        "authorityGroup": request_data["authorityGroup"],
        "contractLockID": "string"  
    }
    
    return {"body": data}


#### 4C - State Context Değer Tutma Scripti

In [1]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    return store


#### 4D - Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "  "}


## Step 5: Closure Accrual Creation
- **State Number**: 5
- **State Code**: 
- **State Name**: Closure Accrual Creation
- **Description**: A closure accrual invoice is generated for the terminated contract. This step is done using the `/accruals/calculate` service.
- **Status**: Pending


## Step 6 -1: Data Preparation for New Account
- **State Number**: -
- **State Code**: -
- **State Name**: Data Preparation for New Account
- **Description**: This process involves preparing the necessary data for creating a new account. Relevant attributes are queried.
- **Status**: Pending

#### 6.1A - Çalışma Öncesi Scripti

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, businessLogic, webhook}>
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):  

    return {}

#### 6.1B - İstek Eşleştirme Scripti

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, businessLogic, webhook}>
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):  
    contractAccountId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractAccountId'),None)['value']

    return {"params": {"assetId":contractAccountId}}

#### 6.1C - State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_response_of_machine):
    store = {'accountInfo' : state_response_of_machine['10380006-1']['businessLogicResponse']}
    return store

#### 6.1D - Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "      "}


## Step 6: Create New Account
- **State Number**: 6
- **State Code**: 
- **State Name**: Create New Account
- **Description**: An account is created to link with the new contract. New account definitions are made using existing attributes.
- **Status**: Pending


#### 6A - Çalışma Öncesi Scripti

In [ ]:
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    return {}

#### 6B - İstek Eşleştirme Scripti 

In [ ]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # Not recursive

    deletelist = ['AssetIdentifier']

    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
 
    copied_json = {}
    stack = [(copied_json, state_responses_of_machine['   X  ']['businessLogicResponse'])]
 
    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()
 
        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value
 
    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959
 
   
    for attr in copied_json['attributes']:
        if attr['attribute']['attributeCode']=='ExpiryDate':
            attr['value'] = current_date

    copied_json['attributes'] = [attr for attr in copied_json['attributes'] if attr['attribute']['attributeCode'] not in deletelist]

   
                   
 
    return {"body":copied_json}  
 


    
    


#### 6C - State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):

    store  = {'NewAccountId' : state_responses_of_machine[' X ']['businessLogicResponse']['id']} 
    
    return store


#### 6D - Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "    X  "}


#### 6E - İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//account/create"""


'http://thor-stage.edas1.com/host/v1//account/create'

## Step 7: Create New Contract
- **State Number**: 7
- **State Code**: 
- **State Name**: Create New Contract
- **Description**: A new contract is created with the new subscriber group. The system uses the same attributes from the existing contract, but the subscriber group is updated.
- **Status**: Pending


#### 7A - Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 7B - İstek Eşleştirme Scripti

In [ ]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # Not recursive 
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    
    changes_dict = {'SubscriberGroup': request_data['NewSubscriberGroup'],
                    'ContractAccountId': context['accountInfo']['id']
                    }
    
    copied_json = {}
    stack = [(copied_json, request_data)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959

    # Update attribute values in context according to the dict above
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['valueDesc'] = None
                    attr['value'] = new_value

    return {"body":copied_json}  

#### 7C - State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):

    store  = {'NewContractId' : state_responses_of_machine[' X ']['businessLogicResponse']['id']} 
    
    return store


##### 7D - Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "END"}


#### 7E - İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//contract/create"""

'http://thor-stage.edas1.com/host/v1//contract/create'